# Fireworks (Five-minute quickstart)

https://materialsproject.github.io/fireworks/quickstart.html

Testing mongodb connection:

In [ ]:
import pymongo
client = pymongo.MongoClient(
    host='mongo',
    username='',
    password='',
    authSource='admin'
)
client.list_database_names()

## Start FireWorks

If not already running, start MongoDB (if your MongoDB is hosted and maintained externally, follow the note below regarding lpad init):


In [ ]:
# !lpad init 

In [ ]:
!cat my_launchpad.yaml

Reset/Initialize the FireWorks database (the LaunchPad):

In [ ]:
! echo 'y' | lpad reset

## Add a Workflow

There are many ways to add Workflows to the database, including a Python API. Let’s start with an extremely simple example that can be added via the command line:

In [ ]:
!lpad add_scripts 'echo "hello"' 'echo "goodbye"' -n hello goodbye -w test_workflow

This added a two-job linear workflow. The first jobs prints hello to the command line, and the second job prints goodbye. We gave names (optional) to each step as “hello” and “goodbye”. We named the workflow overall (optional) as “test_workflow”.

Let’s look at our test workflow:


In [ ]:
!lpad get_wflows -n test_workflow -d more

We get back basic information on our workflows. The second step “goodbye” is waiting for the first one to complete; it is not ready to run because it depends on the first job.


## Run all Workflows

You can run jobs one at a time (“singleshot”) or all at once (“rapidfire”). Let’s run all jobs:

In [ ]:
!rlaunch --silencer rapidfire

Clearly, both steps of our workflow ran in the correct order.

Let’s again look at our workflows:


In [ ]:
!lpad get_wflows -n test_workflow -d more


FireWorks automatically created launcher_ directories for each step in the Workflow and ran them. We see that both steps are complete. Note that there exist options to choose where to run jobs, as well as to tear down empty directories after running jobs.



## Launch the web GUI
If you have a web browser, you can launch the web GUI to see your results:

In [ ]:
# !lpad webgui

Note that there are options to run the web site in a server mode, try lpad webgui -h to see all the options.

open: http://localhost:8888/proxy/5000

## Python code

The following Python code achieves the same behavior:

In [ ]:
from fireworks import Firework, Workflow, LaunchPad, ScriptTask
from fireworks.core.rocket_launcher import rapidfire

# set up the LaunchPad and reset it
launchpad = LaunchPad(host='mongo')
launchpad.reset('', require_password=False)

# create the individual FireWorks and Workflow
fw1 = Firework(ScriptTask.from_str('echo "hello"'), name="hello")
fw2 = Firework(ScriptTask.from_str('echo "goodbye"'), name="goodbye")
wf = Workflow([fw1, fw2], {fw1:fw2}, name="test workflow")

# store workflow and launch it locally
launchpad.add_wf(wf)
rapidfire(launchpad)


In the code above, the `{fw1:fw2}` argument to Workflow is adding a dependency of fw2 to fw1. You could instead define this dependency when defining your FireWorks:

In [ ]:
fw1 = Firework(ScriptTask.from_str('echo "hello"'), name="hello")
fw2 = Firework(ScriptTask.from_str('echo "goodbye"'), name="goodbye", parents=[fw1])
wf = Workflow([fw1, fw2], name="test workflow")